In [1]:
import pandas as pd
import re
import emoji
from pyarabic import araby, normalize

In [2]:
df = pd.read_csv("../../data/row/02_brad_balanced.csv")
if "Unnamed: 0" in df.columns:
    df = df.drop(columns=["Unnamed: 0"])

In [3]:
ARABIC_RANGE = r"\u0600-\u06FF"
non_arabic_pattern   = re.compile(fr"[^{ARABIC_RANGE}\s.!?,؛،؟]+")
elongation_pattern   = re.compile(r"(.)\1{2,}")          # روووعة -> رووعه
repeated_punct_pattern = re.compile(r"([.!?,؛،؟])\1+")

In [4]:
def remove_emojis(text: str) -> str:
    """Remove all emojis from text"""
    return emoji.replace_emoji(text, "")


def normalize_arabic(text: str) -> str:
    """
    Safely normalize Arabic text:
    - remove diacritics (tashkeel)
    - remove tatweel
    - lightly normalize alif variants
    WITHOUT touching ة or hamza positions
    (better for Arabic BERT vocabularies).[web:207][web:215]
    """
    # Remove tashkeel (all diacritics)
    text = araby.strip_tashkeel(text)
    
    # Remove tatweel (ـ)
    text = araby.strip_tatweel(text)
    
    # Light alif normalization (keep hamza attached)
    text = text.replace("إ", "ا").replace("أ", "ا").replace("آ", "ا").replace("ٱ", "ا")
    
    # IMPORTANT: do NOT change ة -> ه and ى -> ي here
    # so words like "رواية" و "عيسى" تبقى مقروءة جيداً لنماذج BERT[web:154]
    return text


def clean_review(text: str) -> str:
    """Clean and normalize Arabic review text"""
    if not isinstance(text, str):
        return ""
    
    # 1) Remove basic noise: URLs, emails, HTML, digits, mentions, hashtags
    text = re.sub(r"<.*?>", " ", text)
    text = re.sub(r"http[s]?://\S+|www\.\S+", " ", text)
    text = re.sub(r"\S+@\S+", " ", text)
    text = re.sub(r"@[A-Za-z0-9_]+", " ", text)
    text = re.sub(r"#[^\s]+", " ", text)
    text = re.sub(r"\d+", " ", text)
    
    # 2) Remove emojis
    text = remove_emojis(text)
    
    # 3) Arabic normalization + remove tashkeel
    text = normalize_arabic(text)
    
    # 4) Keep only Arabic + basic punctuation
    text = non_arabic_pattern.sub(" ", text)
    
    # 5) Collapse elongated letters: روووووعة -> رووعة
    text = elongation_pattern.sub(r"\1\1", text)
    
    # 6) Normalize punctuation and spaces
    text = repeated_punct_pattern.sub(r"\1", text)
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


In [5]:
# -------- apply + filter --------
df["review_clean"] = df["review"].astype(str).apply(clean_review)

In [6]:
MIN_CHARS = 10
before = len(df)
df = df[df["review_clean"].str.len() >= MIN_CHARS].reset_index(drop=True)
after = len(df)
print(f"Removed {before - after} very short reviews (< {MIN_CHARS} chars).")

Removed 2499 very short reviews (< 10 chars).


In [7]:
print(df[["review", "review_clean"]].tail())

                                                   review  \
154002  من الكتب الرائده والأولى في مجال العلاقات الإن...   
154003  كتاب التاو تي تشينغ _ إنجيل الحكمة التاوية في ...   
154004  عجبنى التفاصيل، تفاصيل كتير جدا ودى اكتر حاجة ...   
154005  رغم مشاهدة سلسلة الأفلام، إلا أنه للرواية طعم ...   
154006                           واقعا كيف كردم از خوندنش   

                                             review_clean  
154002  من الكتب الرائده والاولى في مجال العلاقات الان...  
154003  كتاب التاو تي تشينغ انجيل الحكمة التاوية في ال...  
154004  عجبنى التفاصيل، تفاصيل كتير جدا ودى اكتر حاجة ...  
154005  رغم مشاهدة سلسلة الافلام، الا انه للرواية طعم ...  
154006                           واقعا كيف كردم از خوندنش  


In [8]:
df.to_csv("../../data/processed/02_brad_balanced_processed.csv", index=False)
print("Saved: brad_reviews_preprocessed_advanced.csv")

Saved: brad_reviews_preprocessed_advanced.csv
